# Moov AI - Question #1

Ce notebook répond à la question 1 au sujet de la préparation des données



In [21]:
# Import des données et des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go

In [22]:
# Import csv
df = pd.read_csv(r'/Users/philippebeliveau/Desktop/Notebook/Moov AI/stores_sales_forecasting.csv', encoding='ISO-8859-1')

# Transform 'Order Date' and 'Ship Date' to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date']) 

# Transform the postal code in a categorical variable
df['Postal Code'] = df['Postal Code'].astype('str')
df['Row ID'] = df['Row ID'].astype('str')

# Create a feature to compute the difference between the order date and the ship date
df['Order Ship Delta'] = (df['Ship Date'] - df['Order Date']).dt.days

# Create a feature regarding the profit margin 
df['Profit Margin'] = df['Profit'] / df['Sales']

print(f"Shape of the dataset: {df.shape}")
# display(df.head(5).style.set_sticky().set_properties(**{'overflow-x': 'auto'}))

Shape of the dataset: (2121, 23)


In [23]:
# Assure that the data has a rows for every single day
def adjust_dataset_for_daily_entries(df, date_col):
    """Ensures that the dataset has a row for every single day, filling missing days with NaN values."""
    df[date_col] = pd.to_datetime(df[date_col])
    all_dates = pd.date_range(start=df[date_col].min(), end=df[date_col].max(), freq='D')
    
    # Ensure all columns are retained, filling missing values with NaN
    full_df = pd.DataFrame(all_dates, columns=[date_col])
    df = full_df.merge(df, on=date_col, how='left')
    
    return df

df = adjust_dataset_for_daily_entries(df, 'Order Date')

# Question #1 : Préparation des données

Comme c’est souvent le cas dans les projets, le jeu de données peut nécessiter quelques
manipulations pour être utilisable par une approche ML.
- Si tu rencontres des problèmes de qualité des données durant ta manipulation des
données de ventes, comment les as-tu résolus?
- Limite-toi aux trois enjeux les plus pertinents selon toi (appuie-toi avec un visuel).
- Est-ce que les insights trouvés peuvent être transformés en features qui faciliteront
l'apprentissage du modèle M

## Enjeux #1 - Discontinuité (Intermittence)

In [24]:
# Plot my df as a time series, make sure to keep the nan value as is 
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Order Date'], y=df['Sales'], mode='lines', name='Sales'))
fig.update_layout(title='Non-continuous Sales over time', xaxis_title='Date', yaxis_title='Sales')
fig.show()

In [25]:
# Print the number of nan values for the sales column 
print(f"Number of NaN values in the 'Sales' column: {df['Sales'].isna().sum()}")
print(f"Number of days in the dataset: {df['Order Date'].nunique()}")
# Calculate the number of days with missing sales values
missing_sales_days = df['Sales'].isna().sum()
# Calculate the percentage of days with missing sales values
missing_sales_percentage = missing_sales_days / df['Order Date'].nunique() * 100
print(f"Percentage of days with missing sales values: {missing_sales_percentage:.2f}%")

Number of NaN values in the 'Sales' column: 566
Number of days in the dataset: 1455
Percentage of days with missing sales values: 38.90%


Randomness of it 

In [26]:
# Give me the distribution of the days with Nan values, regarding the day of the week 
df['Day of Week'] = df['Order Date'].dt.day_name()
df['Has NaN Sales'] = df['Sales'].isna()
df['Has NaN Sales'] = df['Has NaN Sales'].replace({True: 'Yes', False: 'No'})

fig = px.histogram(df, x='Day of Week', color='Has NaN Sales', barmode='group')
fig.update_layout(title='Distribution of NaN Sales values per day of the week')
fig.show()

### Solution - Enjeux #1

- Deux features décrivant le temps moyen entres deux ventes l’intermittence: Coefficient of Variation (CV) et l’average demand interval (ADI), et ceci selon son niveau de granularité, pourrait aider l’apprentissage du modèle.

In [27]:
import pandas as pd
import numpy as np

def analyze_intermittent_series(data, time_col, sales_col):
    """
    Analyze an intermittent time series by calculating CV, ADI, and Sporadicity Ratio.

    Parameters:
    data (pd.DataFrame): Dataframe containing the time series.
    time_col (str): Column name for the time variable.
    sales_col (str): Column name for the sales variable.

    Returns:
    pd.DataFrame: A summary dataframe with CV, ADI, and Sporadicity Ratio for the series.
    """
    # Sort by time to ensure correct ordering
    data = data.sort_values(by=time_col).reset_index(drop=True)

    # Calculate basic metrics
    total_periods = len(data)
    non_zero_periods = (data[sales_col] > 0).sum()
    avg_demand_interval = total_periods / non_zero_periods if non_zero_periods > 0 else np.nan
    mean_sales = data[sales_col].mean()
    std_sales = data[sales_col].std()

    # Calculate metrics
    cv = std_sales / mean_sales if mean_sales > 0 else np.nan
    adi = avg_demand_interval
    sporadicity_ratio = adi * cv if not (np.isnan(adi) or np.isnan(cv)) else np.nan

    # Create result
    results = {
        "CV (Coefficient of Variation)": cv,
        "ADI (Average Demand Interval)": adi,
        "Sporadicity Ratio": sporadicity_ratio,
    }

    return pd.DataFrame([results])

# Analyze intermittent series
result = analyze_intermittent_series(df, time_col="Order Date", sales_col="Sales")
print(result)


   CV (Coefficient of Variation)  ADI (Average Demand Interval)  \
0                       1.438333                       1.266855   

   Sporadicity Ratio  
0            1.82216  


# Enjeux #2 - Granularité au niveau du magasin 

Le deuxième enjeux attrait au très grand nombre de 0 suite à avoir ramener les séries de manière continue. Et ceci même au niveau le moins granulaire, ce qui veut dire que si nous tentons de filtrer par magasin, nous ne serons pas en mesure d’avoir des séries avec assez d’observations pour prédire adéquatement les ventes futures des magasins. 

Comme on remarque, au niveau journalier, au moment qu’on filtre uniquement sur les régions, les quantités de 0 deviennent astronomique:

In [28]:
def compute_zero_percentage_weekly_by_region(df, date_col, sales_col, region_col):
    """Computes the percentage of zero values at the weekly level for each region."""
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Resampler les données par semaine et par région
    df_weekly = df.set_index(date_col).groupby(region_col).resample('D').agg({
        sales_col: 'sum'
    }).reset_index()
    
    # Compter le nombre total de semaines par région
    total_weeks = df_weekly.groupby(region_col).size()
    
    # Compter les zéros par région
    zero_counts = df_weekly[df_weekly[sales_col] == 0].groupby(region_col).size()
    
    # Calculer le pourcentage de zéros par région
    zero_percentage = (zero_counts / total_weeks) * 100
    
    return zero_percentage

# Exemple d'utilisation
zero_percentage = compute_zero_percentage_weekly_by_region(df, 'Order Date', 'Sales', 'Region')
print(zero_percentage)

Region
Central    77.134986
East       73.793103
South      83.012380
West       68.279198
dtype: float64


Resample weekly

In [29]:

def plot_weekly_sales_trend_by_region(df, date_col, sales_col, region_col):
    """Plots the weekly sales trend by region."""
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Resampler les données par semaine et par région
    df_weekly = df.set_index(date_col).groupby(region_col).resample('D').agg({
        sales_col: 'sum'
    }).reset_index()
    
    # Tracer le graphique
    fig = px.line(df_weekly, x=date_col, y=sales_col, color=region_col, title='Weekly Sales Trend by Region')
    fig.update_layout(xaxis_title='Order Date', yaxis_title='Sales', legend_title='Region')
    fig.show()

# Exemple d'utilisation
plot_weekly_sales_trend_by_region(df, 'Order Date', 'Sales', 'Region')

## Enjeux #3 - Valeurs sporadiques

Cet enjeux concerne les « valeurs aberrantes » sporadiques, qui sont nombreuses au quotidien et qui ne sont pas totalement aléatoires.

In [30]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import zscore

def adjust_dataset_for_daily_entries(df, date_col):
    """Ensures that the dataset has a row for every single day, filling missing days with NaN values."""
    # Convert the date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])

    # Aggregate sales data by day
    daily_data = df.groupby(date_col).sum(numeric_only=True).reset_index()

    # Create a full date range
    all_dates = pd.date_range(start=daily_data[date_col].min(), end=daily_data[date_col].max(), freq='D')

    # Create a DataFrame with the full date range
    full_dates_df = pd.DataFrame(all_dates, columns=[date_col])

    # Merge the full date range with the aggregated daily data
    df = full_dates_df.merge(daily_data, on=date_col, how='left')
    
    return df

def identify_outliers(df, sales_col):
    """Identifies outliers in the sales data using Z-score method."""
    df['Z_Score'] = zscore(df[sales_col].fillna(0))  # Compute Z-scores for sales
    outliers = df[df['Z_Score'].abs() > 3]  # Z-score > 3 indicates an outlier
    return outliers

def assess_holiday_effect(outliers, date_col, holidays):
    """Checks if outliers coincide with known holidays."""
    outliers[date_col] = pd.to_datetime(outliers[date_col])
    holidays = pd.to_datetime(holidays)
    outliers['Is_Holiday'] = outliers[date_col].isin(holidays)
    return outliers

def plot_outliers_with_holidays(df, date_col, sales_col, outliers):
    """Plots sales data and highlights outliers and holidays."""
    fig = px.line(df, x=date_col, y=sales_col, title='Sales with Outliers and Holidays')
    
    # Highlight outliers
    fig.add_trace(go.Scatter(x=outliers[date_col], y=outliers[sales_col], 
                             mode='markers', 
                             marker=dict(color='red', size=8), 
                             name='Outliers'))
    
    # Highlight holidays
    holidays = outliers[outliers['Is_Holiday']]
    fig.add_trace(go.Scatter(x=holidays[date_col], y=holidays[sales_col], 
                             mode='markers', 
                             marker=dict(color='green', size=14), 
                             name='Holidays'))
    
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Sales')
    fig.show()


In [31]:
holidays = [
    '2014-01-01',  # New Year's Day
    '2014-01-20',  # Martin Luther King Jr. Day
    '2014-02-14',  # Valentine's Day (Sales on gifts, chocolates, flowers)
    '2014-02-17',  # Presidents' Day (Major sales on furniture, appliances, cars)
    '2014-03-17',  # St. Patrick's Day (Sales on alcohol, party supplies)
    '2014-04-20',  # Easter Sunday (Sales on candy, clothing, decorations)
    '2014-05-11',  # Mother's Day (Sales on gifts, jewelry, beauty products)
    '2014-05-26',  # Memorial Day (Major sales on home goods, cars, mattresses)
    '2014-06-15',  # Father's Day (Sales on tools, electronics, clothing)
    '2014-07-04',  # Independence Day (Sales on grills, outdoor furniture, appliances)
    '2014-09-01',  # Labor Day (Major sales on furniture, appliances, clothing)
    '2014-10-13',  # Columbus Day (Retail sales, especially clothing and outdoor gear)
    '2014-10-31',  # Halloween (Sales on costumes, candy, decorations)
    '2014-11-11',  # Veterans Day (Military discounts, retail sales)
    '2014-11-27',  # Thanksgiving
    '2014-11-28',  # Black Friday (Biggest shopping day of the year)
    '2014-12-01',  # Cyber Monday (Major online retail discounts)
    '2014-12-25',  # Christmas Day (Post-Christmas sales)
    '2014-12-26',  # Boxing Day (Retail clearance sales)
    
    '2015-01-01',  # New Year's Day
    '2015-01-19',  # Martin Luther King Jr. Day
    '2015-02-14',  # Valentine's Day
    '2015-02-16',  # Presidents' Day
    '2015-03-17',  # St. Patrick's Day
    '2015-04-05',  # Easter Sunday
    '2015-05-10',  # Mother's Day
    '2015-05-25',  # Memorial Day
    '2015-06-21',  # Father's Day
    '2015-07-04',  # Independence Day
    '2015-09-07',  # Labor Day
    '2015-10-12',  # Columbus Day
    '2015-10-31',  # Halloween
    '2015-11-11',  # Veterans Day
    '2015-11-26',  # Thanksgiving
    '2015-11-27',  # Black Friday
    '2015-11-30',  # Cyber Monday
    '2015-12-25',  # Christmas Day
    '2015-12-26',  # Boxing Day

    '2016-01-01',  # New Year's Day
    '2016-01-18',  # Martin Luther King Jr. Day
    '2016-02-14',  # Valentine's Day
    '2016-02-15',  # Presidents' Day
    '2016-03-17',  # St. Patrick's Day
    '2016-03-27',  # Easter Sunday
    '2016-05-08',  # Mother's Day
    '2016-05-30',  # Memorial Day
    '2016-06-19',  # Father's Day
    '2016-07-04',  # Independence Day
    '2016-09-05',  # Labor Day
    '2016-10-10',  # Columbus Day
    '2016-10-31',  # Halloween
    '2016-11-11',  # Veterans Day
    '2016-11-24',  # Thanksgiving
    '2016-11-25',  # Black Friday
    '2016-11-28',  # Cyber Monday
    '2016-12-25',  # Christmas Day
    '2016-12-26',  # Boxing Day

    '2017-01-01',  # New Year's Day
    '2017-01-16',  # Martin Luther King Jr. Day
    '2017-02-14',  # Valentine's Day
    '2017-02-20',  # Presidents' Day
    '2017-03-17',  # St. Patrick's Day
    '2017-04-16',  # Easter Sunday
    '2017-05-14',  # Mother's Day
    '2017-05-29',  # Memorial Day
    '2017-06-18',  # Father's Day
    '2017-07-04',  # Independence Day
    '2017-09-04',  # Labor Day
    '2017-10-09',  # Columbus Day
    '2017-10-31',  # Halloween
    '2017-11-11',  # Veterans Day
    '2017-11-23',  # Thanksgiving
    '2017-11-24',  # Black Friday
    '2017-11-27',  # Cyber Monday
    '2017-12-25',  # Christmas Day
    '2017-12-26',  # Boxing Day
]


In [32]:
# Convert the holiday dates to datetime if needed
holidays = pd.to_datetime(holidays)

df = adjust_dataset_for_daily_entries(df, 'Order Date')
outliers = identify_outliers(df, 'Sales')
outliers_with_holidays = assess_holiday_effect(outliers, 'Order Date', holidays)
plot_outliers_with_holidays(df, 'Order Date', 'Sales', outliers_with_holidays)

/var/folders/dj/q5s8lngd1msdbb76w1tn9x1r0000gn/T/ipykernel_19843/1856354192.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dj/q5s8lngd1msdbb76w1tn9x1r0000gn/T/ipykernel_19843/1856354192.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import zscore

def identify_outliers(df, sales_col):
    """Creates a new column indicating if a row is an outlier based on Z-scores."""
    df['Z_Score'] = zscore(df[sales_col].fillna(0))  # Compute Z-scores for sales
    df['Is_Outlier'] = df['Z_Score'].abs() > 3  # Outliers have Z-score > 3
    return df

def outlier_distribution_by_weekday(df, date_col, outlier_col):
    """Analyzes the distribution of outliers across days of the week."""
    df[date_col] = pd.to_datetime(df[date_col])
    df['Day_of_Week'] = df[date_col].dt.day_name()  # Add day of week column
    outlier_distribution = df[df[outlier_col]].groupby('Day_of_Week').size().reset_index(name='Count')
    
    # Sort by day of the week order
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    outlier_distribution['Day_of_Week'] = pd.Categorical(outlier_distribution['Day_of_Week'], categories=days_order, ordered=True)
    outlier_distribution = outlier_distribution.sort_values('Day_of_Week')
    
    # Plot the distribution
    fig = px.bar(outlier_distribution, x='Day_of_Week', y='Count', title='Outlier Distribution by Day of the Week', 
                 labels={'Count': 'Number of Outliers', 'Day_of_Week': 'Day of the Week'})
    fig.show()
    
    return outlier_distribution

# Example usage
df = adjust_dataset_for_daily_entries(df, 'Order Date')
df = identify_outliers(df, 'Sales')
outlier_distribution = outlier_distribution_by_weekday(df, 'Order Date', 'Is_Outlier')


In [34]:
import pandas as pd
import plotly.express as px
from scipy.stats import zscore

def identify_outliers(df, sales_col):
    """Creates a new column indicating if a row is an outlier based on Z-scores."""
    df['Z_Score'] = zscore(df[sales_col].fillna(0))  # Compute Z-scores for sales
    df['Is_Outlier'] = df['Z_Score'].abs() > 3  # Outliers have Z-score > 3
    return df

def outlier_distribution_by_month_year(df, date_col, outlier_col):
    """Analyzes the distribution of outliers across months and years."""
    df[date_col] = pd.to_datetime(df[date_col])
    df['Month'] = df[date_col].dt.month_name()  # Add month column
    df['Year'] = df[date_col].dt.year  # Add year column
    outlier_distribution = df[df[outlier_col]].groupby(['Year', 'Month']).size().reset_index(name='Count')
    
    # Sort by month order
    months_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    outlier_distribution['Month'] = pd.Categorical(outlier_distribution['Month'], categories=months_order, ordered=True)
    outlier_distribution = outlier_distribution.sort_values(['Year', 'Month'])
    
    # Plot the distribution
    fig = px.bar(outlier_distribution, x='Month', y='Count', color='Year', barmode='group', 
                 title='Outlier Distribution by Month and Year', 
                 labels={'Count': 'Number of Outliers', 'Month': 'Month', 'Year': 'Year'})
    fig.show()
    
    return outlier_distribution

# Example usage
df = identify_outliers(df, 'Sales')
outlier_distribution = outlier_distribution_by_month_year(df, 'Order Date', 'Is_Outlier')

## Conclusion

La difficulté des données se reflète dans l'intermittence de la série et dans le fait que les directeurs souhaitent recevoir une prédiction au niveau du magasin, ce qui les incite à faire une prédiction granulaire, qui à son tour génère plus de 0. Comme le test ne décrit pas ce qui constitue un magasin, j'émettrai l'hypothèse qu'un « magasin » peut être décrit par une région et que c'est par région que les ventes futures seront prédites.